# VOLTTRON Reference App

This notebook installs VOLTTRON agents and other software that simulates participation in Open ADR (Automated Demand Response). 

In this simulation, an OpenADR Virtual Top Node (VTN) sends ADR event information to a VOLTTRON Virtual End Node (VEN) agent. The VEN transmits the event data to a ReferenceApp agent that controls the simulation. The ReferenceAppAgent responds by adjusting the behavior of a set of simulated device drivers. Report information is transmitted from the ReferenceAppAgent back to the VEN, which relies it to the VTN. Meanwhile, a Historian agent captures device metrics. The VOLTTRON Central web user interface graphs the device telemetry, and the VTN graphs ADR event telemetry.

Further information about VOLTTRON's OpenADR agent is available [here](https://volttron.readthedocs.io/en/develop/core_services/openadr).

Much of this notebook's setup and execution is done using shell commands, called from Python.

# Setup: Prepare the VOLTTRON Environment and Jupyter Server

VOLTTRON must be installed before using this notebook. For detailed instructions on
installing and configuring a VOLTTRON/Jupyter server environment, 
see [Jupyter Notebooks](http://volttron.readthedocs.io/en/develop/devguides/supporting/JupyterNotebooks.html) 
in VOLTTRON ReadTheDocs.

As is described in that guide, environment variables should have been defined before starting 
the Jupyter server:

````
$ export VOLTTRON_ROOT=~/repos/volttron
````
        (path of the VOLTTRON repository, installed prior to running bootstrap)

````
$ export VOLTTRON_HOME=~/.volttron
````
        (directory in which the VOLTTRON instance runs)

````
$ cd $VOLTTRON_ROOT
$ source env/bin/activate
````
        (activate the VOLTTRON virtual environment)

````
$ jupyter notebook
````
        (start the Jupyter notebook server)
        
The Python code below does some initialization to prepare for the steps that follow.

In [ ]:
import datetime
import json
import os
import pprint
import sqlite3
import subprocess
import sys
import time

# Define a "run this shell command" method, wrapping subprocess.check_output()
def _sh(shell_command, shell=True, stderr=None):
    try:
        return_value = subprocess.check_output(shell_command, shell=shell, stderr=stderr)
    except Exception, err:
        print('Shell command failed: {}', shell_command)
        print(err)
        return_value = 'Error'
    return return_value

# Same as _sh(), except that this also prints the command output, preceded by an optional label.
def _print_sh(shell_command, label=None, **kwargs):
    print('{0}: {1}\n'.format(label+':' if label else '', _sh(shell_command, **kwargs)))

# Set up local variables vhome and vroot.
# The environment variables VOLTTRON_ROOT and VOLTTRON_HOME should already be defined -- see above.
vroot = %env VOLTTRON_ROOT
vhome = %env VOLTTRON_HOME
print("VOLTTRON_ROOT={}".format(vroot))
print("VOLTTRON_HOME={}".format(vhome))

# Define a VIP_SOCKET environment variable for use while installing and running agents.
socket_name = 'ipc://' + vhome + '/run/vip.socket'
%env VIP_SOCKET=$socket_name

# Run from the VOLTTRON root directory.
os.chdir(vroot)

print("Initialization complete")

# Setup: Prepare the Simulation Environment

The simulation software resides in a separate repository, volttron-applications.

The repository must be downloaded from github, creating a new directory parallel to $VOLTTRON_ROOT, as follows:

````
$ cd $VOLTTRON_ROOT
$ cd ..
$ git clone git://github.com/VOLTTRON/volttron-applications.git
````

Then a symbolic link named "applications" should be added under $VOLTTRON_ROOT:

````
$ cd $VOLTTRON_ROOT
$ ln -s ../volttron-applications/ applications
````

# Setup: Install and Start the OpenADR VTN Server

Because a core component of the simulation is the VEN agent's communication with the VTN, the VTN (which is a separate server, not a VOLTTRON agent) must be installed and started. The documentation for such installation can be found [here](https://volttron.readthedocs.io/en/latest/core_services/openadr/VtnServerConfig.html).

# Setup: Shut Down All Agents

This ensures a clean agent installation process by the notebook.

In [ ]:
print('Wait for the list to be displayed, and confirm that no agents are listed as running...\n')

# Shut down all agents.
_sh('volttron-ctl shutdown')

# List agent status to verify that the status of each agent is 0 or blank.
_print_sh('volttron-ctl status', stderr=subprocess.STDOUT)

# Setup: Install Agents

Install each agent that will be employed by the simulation: 

* A ReferenceApp agent
* An OpenADR VEN agent
* A simulation clock agent
* A simulation driver agent
* An actuator agent
* A VolttronCentral agent
* A VolttronCentralPlatform agent
* A platform historian agent

In [ ]:
print('Wait for the list to be displayed, then confirm that all of the agents appear in it...')

def install_agent(dir=None, id=None, config=None, tag=None):
    script_install_command = 'python scripts/install-agent.py -s {0} -i {1} -c {2} -t {3} -f'
    _sh(script_install_command.format(dir, id, config, tag))
    print('Installed {}'.format(tag))

# Install the SimulationDriver Agent
agent_root = vroot + '/applications/kisensum/Simulation/SimulationDriverAgent'
install_agent(dir=agent_root,
              id='simulation.driver',
              config=agent_root+'/simulationdriver.config',
              tag='simulation.driver')

# Install the SimulationClock Agent
agent_root = vroot + '/applications/kisensum/Simulation/SimulationClockAgent'
install_agent(dir=agent_root,
              id='simulationclock',
              config=agent_root+'/simulationclock.config', 
              tag='simulationclock')

# Install the ReferenceApp Agent
agent_root = vroot + '/applications/kisensum/ReferenceAppAgent'
install_agent(dir=agent_root,
              id='referenceappagent',
              config=agent_root+'/referenceappagent.config', 
              tag='referenceappagent')

# Install an OpenADR VEN Agent
agent_root = vroot + '/services/core/OpenADRVenAgent'
install_agent(dir=agent_root,
              id='venagent',
              config=agent_root+'/openadrven.config',
              tag='venagent')

# Install a Volttron Central Platform Agent
agent_root = vroot + '/services/core/VolttronCentralPlatform'
install_agent(dir=agent_root,
              id='platform.agent',
              config=vroot+'/applications/kisensum/ReferenceAppAgent/vcp_simulation.config', 
              tag='vcp')

# Install a Volttron Central Agent
agent_root = vroot + '/services/core/VolttronCentral'
install_agent(dir=agent_root,
              id='volttron.central',
              config=agent_root+'/config', 
              tag='vc')

# Install an Actuator Agent
agent_root = vroot + '/services/core/ActuatorAgent'
install_agent(dir=agent_root,
              id='simulation.actuator',
              config=vroot+'/applications/kisensum/ReferenceAppAgent/actuator.config',
              tag='simulation.actuator')

# Install a Platform SQLHistorian Agent
agent_root = vroot + '/services/core/SQLHistorian'
config = """{{
    "agentid": "sqlhistorian-sqlite",
    "connection": {{
        "type": "sqlite",
        "params": {{
            "database": "{0}/data/platform.historian.sqlite"
        }}
    }}
}}""".format(vhome)
with open(vhome+'/platform_historian.config', 'w') as file:
    file.write(config)
install_agent(dir=agent_root,
              id='platform.historian',
              config=vhome+'/platform_historian.config',
              tag='platform_historian')

# List agent status to verify that the agents were installed successfully.
_print_sh('volttron-ctl status', stderr=subprocess.STDOUT)

# Setup: Install Simulation Device Drivers

The simulation relies on device drivers that simulate building load, solar power and battery charging/discharging. Further information on the simulation subsystem is available [here](http://volttron.readthedocs.io/en/develop/volttron_applications/Simulated-Drivers.html).

In [ ]:
print('Wait for the simulation driver configs to be displayed, then confirm that all of these configs appear in it...')

driver_root = vroot + '/applications/kisensum/Simulation/SimulationDriverAgent/'

def install_driver_csv(name=None, csv=None):
    _sh('volttron-ctl config store simulation.driver {0} {1} --csv'.format(name, driver_root + csv))

def install_driver_config(name=None, config=None):
    _sh('volttron-ctl config store simulation.driver {0} {1}'.format(name, driver_root + config))
    
# Install simload, the simulated load driver
install_driver_csv(name='simload.csv', csv='simload.csv')
install_driver_config(name='devices/campus1/building1/simload', config='simload.config')

# Install simpv, the simulated PV driver
install_driver_csv(name='simpv.csv', csv='simpv.csv')
install_driver_config(name='devices/campus1/building1/simpv', config='simpv.config')

# Install simstorage, the simulated storage driver
install_driver_csv(name='simstorage.csv', csv='simstorage.csv')
install_driver_config(name='devices/campus1/building1/simstorage', config='simstorage.config')

# List the Simulation Driver configuration to confirm that the drivers were installed successfully.
_print_sh('volttron-ctl config list simulation.driver')

# Execution: Refresh Variables and Stop Agents

Before starting the agents, refresh all variables and make sure that all agents are stopped.

In [ ]:
print('Make a fresh start - refresh variable definitions, shut down any running agents, refresh the database')

import datetime
import json
import os
import pprint
import sqlite3
import subprocess
import sys
import time

# Define a "run this shell command" method, wrapping subprocess.check_output()
def _sh(shell_command, shell=True, stderr=None):
    try:
        return_value = subprocess.check_output(shell_command, shell=shell, stderr=stderr)
    except Exception, err:
        print('Shell command failed: {}', shell_command)
        print(err)
        return_value = 'Error'
    return return_value

# Same as _sh(), except that this also prints the command output, preceded by an optional label.
def _print_sh(shell_command, label=None, **kwargs):
    print('{0}: {1}\n'.format(label+':' if label else '', _sh(shell_command, **kwargs)))

# Set up local variables vhome and vroot.
# The environment variables VOLTTRON_ROOT and VOLTTRON_HOME should already be defined -- see above.
vroot = %env VOLTTRON_ROOT
vhome = %env VOLTTRON_HOME
print("VOLTTRON_ROOT={}".format(vroot))
print("VOLTTRON_HOME={}".format(vhome))

# Define a VIP_SOCKET environment variable for use while installing and running agents.
socket_name = 'ipc://' + vhome + '/run/vip.socket'
%env VIP_SOCKET=$socket_name

# Run from the VOLTTRON root directory.
os.chdir(vroot)

# Shut down all agents.
_sh('volttron-ctl shutdown')

# List agent status to verify that the status of each agent is 0 or blank.
_print_sh('volttron-ctl status', stderr=subprocess.STDOUT)

# Execution: Start Agents

When ready to start the simulation and interact with the VTN, start the agents.

In [ ]:
print('Wait for the list to be displayed, then confirm that each started agent is running...')

_sh('volttron-ctl start --tag vcp')
_sh('volttron-ctl start --tag vc')
_sh('volttron-ctl start --tag platform_historian')
_sh('volttron-ctl start --tag simulationclock')
_sh('volttron-ctl start --tag simulation.driver')
_sh('volttron-ctl start --tag simulation.actuator')
_sh('volttron-ctl start --tag venagent')
_sh('volttron-ctl start --tag referenceappagent')


# List agent status to verify that the started agents have status "running".
_print_sh('volttron-ctl status', stderr=subprocess.STDOUT)

# Use VOLTTRON Central to Observe Device Metrics

Run VOLTTRON Central in a browser to report on simulated device metrics. Information on interacting with the browser interface is available [here](http://volttron.readthedocs.io/en/develop/core_services/service_agents/central_management/VOLTTRON-Central.html).

# Set Up an Event in the OpenADR VTN

Following the guidelines in 
the [OpenADR User Guide](http://volttron.readthedocs.io/en/develop/core_services/openadr/VtnServerGuide.html), 
use the VTN to create an event and view a graph of the event's reported power metrics.